In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
feature_x_sample_processed = pd.read_table(
    PATH_DICT["feature_x_sample.processed.tsv"], index_col=0
)

In [ ]:
for element_type in ("feature", "sample"):

    if element_type == "feature":

        element_alias = FEATURE_ALIAS

        df = feature_x_sample_processed

    elif element_type == "sample":

        element_alias = SAMPLE_ALIAS

        df = feature_x_sample_processed.T

    element_x_fit_parameter = ccal.fit_skew_t_pdfs(
        df,
        n_job=MAX_N_JOB,
        output_file_path=PATH_DICT["{}_x_fit_parameter.tsv".format(element_type)],
    )

    element_x_fit_parameter = pd.read_table(
        PATH_DICT["{}_x_fit_parameter.tsv".format(element_type)], index_col=0
    )

    element_x_fit_parameter["log(Degree of Freedom)"] = ccal.log_nd_array(
        element_x_fit_parameter["Degree of Freedom"]
    )

    for parameter, parameter_values in element_x_fit_parameter.drop(
        "Degree of Freedom", axis=1
    ).items():

        value_name = "{} {}".format(element_alias, parameter)

        ccal.plot_distributions(
            (parameter_values,),
            names=(value_name,),
            plot_rug=parameter_values.size < PLOT_MAX_SIZE ** 0.5,
            title="Distribution of {}".format(value_name),
            xaxis_title=value_name,
        )

        parameter_values_sorted = parameter_values.sort_values()

        first_element = parameter_values_sorted.index[0]

        last_element = parameter_values_sorted.index[-1]

        ccal.plot_distributions(
            (df.loc[first_element], df.loc[last_element]),
            names=(first_element, last_element),
            plot_rug=df.shape[1] < PLOT_MAX_SIZE ** 0.5,
            title="Distributions of {} with Extreme {}".format(
                element_alias, parameter
            ),
            xaxis_title=FEATURE_X_SAMPLE_VALUE_NAME,
        )

        n_extreme = 3

        for rank in list(range(n_extreme)) + list(range(-n_extreme, 0)):

            ranked_element = parameter_values_sorted.index[rank]

            element_values = df.loc[ranked_element]

            n_data, location, scale, degree_of_freedom, shape = element_x_fit_parameter.loc[
                ranked_element,
                ["N Data", "Location", "Scale", "Degree of Freedom", "Shape"],
            ]

            ccal.plot_context(
                element_values,
                n_data=n_data,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                plot_rug=element_values.dropna().size < PLOT_MAX_SIZE ** 0.5,
                title="{} Rank {}: {}".format(value_name, rank, ranked_element),
                xaxis_title=FEATURE_X_SAMPLE_VALUE_NAME,
            )